In [ ]:
!sudo apt-get install libudunits2-dev


In [1]:
install.packages(c('ggmap','sf','pht','odbc'))


Installing packages into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘aws.signature’, ‘RgoogleMaps’, ‘units’, ‘aws.s3’




In [2]:
library(ggplot2)
library(ggmap)
library(sf)

library(dplyr)

library(pht)

library(DBI)
library(odbc)

Google's Terms of Service: https://cloud.google.com/maps-platform/terms/.

Please cite ggmap if you use it! See citation("ggmap") for details.

Linking to GEOS 3.6.2, GDAL 2.2.3, PROJ 4.9.3


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [3]:
library(getPass)

In [4]:
# Load geodata -----------------------------------------------------------------

# Connect to EDW and query for data

conn <- DBI::dbConnect(
  odbc::odbc(),
  dsn = "EDWP_USER",
  uid = getPass::getPass("UID: "),
  pwd = getPass::getPass("PASSWORD: ")
)

UID: ········
PASSWORD: ········


In [ ]:
sql_file <- "member_geo.sql"

In [ ]:
# Expecting these fields (named exactly):
# lat
# lon
map_geo <-
  readChar(sql_file, file.info(sql_file)$size) %>%
  strsplit(";", fixed = TRUE) %>%
  .[[1]] %>%
  lapply(., function(x) DBI::dbGetQuery(conn, x)) %>%
  .[sapply(., nrow) > 0] %>%
  .[[1]]

DBI::dbDisconnect(conn)

In [ ]:
map_geo <- sf::st_as_sf(map_geo, coords = c("lon", "lat"), remove = FALSE)

In [ ]:
# Transform data ---------------------------------------------------------------

# Create the Walmart feature.
walmart <- sf::st_as_sf(data.frame(x = -90.13230, y = 32.40148),
                        coords = c("x", "y"),
                        remove = FALSE)
st_crs(walmart) <- 4326

In [ ]:
# Make a map -------------------------------------------------------------------

# Provide some buffer around the edges of the map
bbox <- unname(st_bbox(map_geo) + c(-.03, -.03, .03, .03))

In [ ]:
map_tiles <- 
  ggmap::get_stamenmap(bbox = bbox, zoom = 12, maptype = "toner-hybrid")

# Population density
ggmap::ggmap(map_tiles) +
  geom_point(
    data = map_geo,
    aes(x = lon, y = lat),
    size = .4,
    shape = 16,
    color = "#005986",
    fill = "#005986",
    alpha = .5) +
  geom_point(
    data = walmart,
    aes(x = x, y = y),
    size = 2,
    shape = 21,
    color = "#222222",
    fill = "#cf1d67",
    alpha = 1) +
  pht::ph_map_theme()

ggsave("vax_outreach_map.png")